# 1 introduction
<br>
This notebook compute the OWT areas in a time series in the Curuai lake

# 2 Functions

In [1]:
# library used
from osgeo import gdal
from osgeo.gdalconst import *
import os
def image_to_arrays(image):
    '''
    This function read a single band image as numpy array
    -----------------------
    image(str): image path
    -----------------------
    return: the five bands necessary for the OWT classification.

    '''
    
    # abre a imagem
    image =  gdal.Open(image, GA_ReadOnly)
    
    # abre as bandas
    array = image.ReadAsArray()

    return array

def array_to_image(img, i, path, geotransform, projection):
    '''
    This function convert a array to tiff
    -----------------------------
    img (np.array): an image array.
    i (str): image name.
    path: path for saving the tiff image.
    geotransform: Geotransform for saving the image (e.g.: image.GetGeoTransform()).
    projection: Projection for saving the image (e.g.: image.GetProjection())
    '''   
    # save results
    # set image dimensions
    image_size = img.shape
    nx = image_size[0]
    ny = image_size[1]

    # create the classified raster file
    dst_ds = gdal.GetDriverByName('GTiff').Create(path+i, ny, nx, 1, gdal.GDT_Int16)

    dst_ds.SetGeoTransform(geotransform)    # specify coords
    dst_ds.SetProjection(projection) # export coords to file       
    dst_ds.GetRasterBand(1).WriteArray(img)   # write r-band to the raster
    dst_ds.GetRasterBand(1).SetNoDataValue(0) 
    dst_ds.FlushCache()                     # write to disk


# 3 Calculate OWT frequency

In [2]:
# library used
import glob
from rasterstats import zonal_stats
import datetime as dt
import pandas as pd

img_list = glob.glob("00_Database/03_Images/03_Funil/l3_owts/*.tif")

# extract values
img_arrays = {}
for img in img_list:
    
    
    date = img.split('/')[-1].split('.')[0].split('_')[1][0:8]
    year = int(date[0:4])
    month = int(date[4:6])
    day = int(date[6:8])
    date = dt.date(year, month, day)
    img_array = image_to_arrays(img)
    img_arrays[date] = img_array

In [3]:
# library used
import numpy as np

# compute OWT frequency
owt_freq = {}
for x in range(0,9):
    
    owt_freq[x] = np.zeros((5490,5490))


    for y in img_arrays.keys():
    
        img_array = img_arrays[y]==x
        owt_freq[x] = owt_freq[x]+img_array
        
    owt_freq[x] = (owt_freq[x]/len(img_arrays.keys()))*100
    owt_freq[x] = np.nan_to_num(owt_freq[x])

# 4 Saves the frequency images

In [4]:
# extract projection data
img_ref = gdal.Open(img)
geotransform = img_ref.GetGeoTransform()
proj = img_ref.GetProjection()

# saves all images
for x in owt_freq.keys():
    
    array_to_image(owt_freq[x],
                   'OWT_'+str(x)+'_frequency.tif',
                   '00_Database/03_Images/03_Funil/l3_owts_frequency/',
                   geotransform,
                   proj)